In [ ]:
import numpy as np 
import pandas as pd 

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
import re, string, unicodedata
from string import punctuation



# Text Classification Using Bernoulli Naive Bayes algorithm

In [ ]:
nltk.download('stopwords')
sw = stopwords.words("english")
training = pd.read_csv('tweet_dataset.csv')

# training = training[:10000]

training.isnull().sum()
training=training.dropna()
training.loc[training["sentiment"] == "empty", "sentiment"] = "neutral"
training.loc[training["sentiment"] == "fun", "sentiment"] = "happiness"
training.loc[training["sentiment"] == "enthusiasm", "sentiment"] = "happiness"
training.loc[training["sentiment"] == "hate", "sentiment"] = "anger"
training["sentiment"].value_counts()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


neutral      6450
worry        5910
happiness    5437
sadness      3577
love         2670
surprise     1548
relief       1055
anger         990
boredom       130
Name: sentiment, dtype: int64

In [ ]:
total = training.isnull().sum().sort_values(ascending=False)
percent = (training.isnull().sum()/training.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
textID,0,0.0
sentiment,0,0.0
author,0,0.0
text,0,0.0
old_text,0,0.0
aux_id,0,0.0
new_sentiment,0,0.0
selected_text,0,0.0


In [ ]:
training.sentiment = pd.Categorical(pd.factorize(training.sentiment)[0])
training.head()


,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text
1,1956967666,0,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...,c811396dc2,negative,headache
2,1956967696,0,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,9063631ab1,negative,gloomy
3,1956967789,1,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,2a815f151d,positive,wants to hang out with friends SOON!
4,1956968416,2,xkilljoyx,We want to trade with someone who has Houston...,@dannycastillo We want to trade with someone w...,82565a56d3,neutral,We want to trade with someone who has Houston ...
5,1956968477,3,xxxPEACHESxxx,Re-pinging : why didn`t you go to prom? BC my ...,Re-pinging @ghostridah14: why didn't you go to...,a610d6b25b,negative,didn`t like my


In [ ]:
def hapus_url(text):
    return re.sub(r'http\S+','', text)
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-Z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
def stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text=' '.join([ps.stem(word) for word in text.split()])
    return text

def final_clean(text):
    final_text= []
    for i in text.split():
        if i.strip().lower() not in sw and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)
  

In [ ]:
def clean(text):
    text = hapus_url(text)
    text = remove_special_characters(text, remove_digits=True)
    text = stemmer(text)
    text = final_clean(text)
    return text

In [ ]:
training['text'] = training['text'].apply(clean)

In [ ]:
%matplotlib inline
training.columns

Index(['textID', 'sentiment', 'author', 'text', 'old_text', 'aux_id',
       'new_sentiment', 'selected_text'],
      dtype='object')

In [ ]:
training.drop(['textID'], axis=1, inplace=True)

In [ ]:
print(training["old_text"])

1        Layin n bed with a headache  ughhhh...waitin o...
2                      Funeral ceremony...gloomy friday...
3                     wants to hang out with friends SOON!
4        @dannycastillo We want to trade with someone w...
5        Re-pinging @ghostridah14: why didn't you go to...
                               ...                        
39992    @jasimmo Ooo showing of your French skills!! l...
39996                       Happy Mothers Day  All my love
39997    Happy Mother's Day to all the mommies out ther...
39998    @niariley WASSUP BEAUTIFUL!!! FOLLOW ME!!  PEE...
39999    @mopedronin bullet train from tokyo    the gf ...
Name: old_text, Length: 27767, dtype: object


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

tfidf_vect = TfidfVectorizer()


text1 = tfidf_vect.fit_transform(training["old_text"])
y = training["sentiment"]

X_train, X_test,y_train,y_test = train_test_split(text1, y, test_size=0.2, random_state=None)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

print(X_train.shape)

(22213, 37220)


In [ ]:
model = SVC()
model.fit(X_train, y_train)
print(model)

SVC()


In [ ]:
pred_svm = model.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(pred_svm,y_test)

0.38080662585523944

In [ ]:
print(y_test)

15325    8
34637    3
32108    2
1917     2
28909    1
        ..
35572    2
30629    1
30726    2
26005    3
26295    5
Name: sentiment, Length: 5554, dtype: category
Categories (9, int64): [0, 1, 2, 3, ..., 5, 6, 7, 8]


In [ ]:
print(pred_svm)

[0 1 2 ... 3 2 2]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 500, analyzer='word')

cv_addr = cv.fit_transform(training['sentiment'])

x = pd.SparseDataFrame(cv_addr, columns=cv.get_feature_names(), 
                                index=training.index, default_fill_value=0)

x
training.join(x)

AttributeError: ignored